# Artificial Neural Network in Pytorch

In [1]:
#Importing libraries
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Load dataset
df = pd.read_csv("Churn_Modelling.csv")

# Select relevant features (excluding row number, customer ID, and surname)
X = df.iloc[:, 3:13].values
y = df.iloc[:, 13].values  # Target variable

# Encode categorical variables (Gender and Geography)
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])  # Encode Gender

# One-hot encode Geography
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)


# Convert data to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Scale the features
scaler = StandardScaler()
X_train_tensor = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_test_tensor = torch.tensor(scaler.transform(X_test), dtype=torch.float32)


In [3]:
# Define the Neural Network
class ChurnModel(nn.Module):
    def __init__(self):
        super(ChurnModel, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 6)  # First hidden layer
        self.fc2 = nn.Linear(6, 6)  # Second hidden layer
        self.fc3 = nn.Linear(6, 1)  # Output layer
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()  # Binary classification

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Create model instance
model = ChurnModel()


In [4]:
criterion = nn.BCELoss()  # Binary Cross-Entropy for classification
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam optimizer


In [5]:
num_epochs = 100
batch_size = 32
dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()  # Reset gradients
        y_pred = model(batch_X)  # Forward pass
        loss = criterion(y_pred, batch_y)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [10/100], Loss: 0.5612
Epoch [20/100], Loss: 0.4619
Epoch [30/100], Loss: 0.3571
Epoch [40/100], Loss: 0.2990
Epoch [50/100], Loss: 0.4514
Epoch [60/100], Loss: 0.2571
Epoch [70/100], Loss: 0.4682
Epoch [80/100], Loss: 0.3213
Epoch [90/100], Loss: 0.3075
Epoch [100/100], Loss: 0.5348


In [6]:
# Make predictions
with torch.no_grad():
    y_pred_prob = model(X_test_tensor)  # Probability predictions
    y_pred = (y_pred_prob > 0.5).float()  # Convert to binary (0 or 1)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred.numpy())
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.8265
